# Get Top Terms by Job Type
*David Norrish, October 2019*

Using the trained phrase models, we can now compare different job types (data science vs data engineering vs control jobs) and see which phrases are most uplifted in each.

In [1]:
from gensim.models.phrases import Phrases, Phraser
import pandas as pd
from pandas import DataFrame

import spacy

from utils import nlp, PARAMS

Read in all jobs CSVs, load bigram and trigram models and derive normed and phrased texts.

## 1. Normalise and phrase model texts
Read in the job description files for data science, data engineering and control jobs. Normalise the text then fit the trigram model.

In [2]:
# Put DataFrames in a dictionary
dfs = {str(path.stem[:-5]): pd.read_csv(path) for path in PARAMS['JOB_PATHS']}
dfs.keys()

dict_keys(['data_engineer', 'control', 'data_scientist'])

In [3]:
bigram_model = Phraser(Phrases.load(str(PARAMS['BIGRAM_MODEL_PATH'])))
trigram_model = Phraser(Phrases.load(str(PARAMS['TRIGRAM_MODEL_PATH'])))

Steps in the text cleaning:

1. Start with full job description text
2. Convert to spaCy-parsed doc
3. Lemmatize to list of spacy.Token lower-case lemmas (with punctuation marks retained)
  4. Save as space-separated string of tokens
5. Apply phrase model to get list of string terms/phrases
6. Drop punctuation tokens
  - Currently done by joining phrase tokens into space-separated string and re-spaCy-parsing. Seems fairly inefficient, could probably just check for punctuation tokens, but non-trivial not to addidentally exclude valid tokens

In [4]:
def add_normed_text(df: DataFrame, col: str):
    """Normalize text by lematizing"""
    normed_texts = []
    for doc in nlp.pipe(df[col]):
        cleaned_toks = [tok.lemma_.lower() for tok in doc if not tok.is_space]
        normed_texts.append(' '.join(cleaned_toks))
    df['normed_text'] = normed_texts
    return df

def add_phrased_text(df: DataFrame, col: str):
    """
    Apply phrase model to join frequently co-occurring tokens.

    Carry out in 2 passes: First retain all punctuation to fit phrase model the
    same as was trained. In a second pass drop punctuation, as just want
    term counts not punctuation.
    """
    phrased_texts = []
    for text in df[col]:
        cleaned_toks = text.split(' ')
        phrased_texts.append(' '.join(trigram_model[bigram_model[cleaned_toks]]))
    cleaned_phrased = []
    for doc in nlp.pipe(phrased_texts):
        cleaned_phrased.append(' '.join([tok.text for tok in doc if not tok.is_punct]))
    df['phrased_text'] = cleaned_phrased
    return df

In [5]:
for path, df in dfs.items():
    print(f"Cleaning {path} jobs...")
    df = add_normed_text(df, 'text')
    df = add_phrased_text(df, 'normed_text')
    output_path = PARAMS['CLEANED_PATH'] / f"{path}_cleaned.csv"
    df.to_csv(output_path, index=None)

Cleaning data_engineer jobs...
Cleaning control jobs...
Cleaning data_scientist jobs...


In [6]:
dfs['data_scientist'].sample(5)

,title,institution,date,text,required skills,applicants skills,normed_text,phrased_text
6,Data Scientist,GHD,27/09/2019,GHD Digital is rapidly growing & are looking for that special person who can help drive the deli...,NaN,NaN,ghd digital be rapidly grow & be look for that special person who can help drive the delivery of...,ghd_digital be rapidly grow be_look_for that special person who_can help drive the delivery of t...
36,Data Scientist,fibreHR,30/09/2019,Bring your love of data science to this role\n Exciting time of growth\n Collaborative...,NaN,NaN,bring -pron- love of datum science to this role exciting time of growth collaborative and suppor...,bring -pron- love of datum_science to this_role exciting time of growth collaborative and suppor...
24,Data Scientist,Mane Consulting,28/09/2019,"Highly regarded, Global specilaist Data and Analytics solutions company seeks a Data Scientist p...",NaN,NaN,"highly regard , global specilaist data and analytics solution company seek a data scientist pass...",highly_regard global specilaist data and analytics solution company seek a data_scientist passio...
31,Data Scientist,Capgemini,14/09/2019,"A global leader in consulting, technology services and digital transformation, the Capgemini Gro...",NaN,"Python (Programming Language), Machine Learning, SQL, Data Analysis, R, Data Science, Microsoft ...","a global leader in consulting , technology service and digital transformation , the capgemini gr...",a global leader in consulting technology service and digital_transformation the capgemini group ...
37,Data Scientist - State Government,Hudson,30/09/2019,Data Scientist - 9 months contract role\n\n 9 month contract role within a key state Gov\n ...,NaN,NaN,data scientist - 9 month contract role 9 month contract role within a key state gov child inform...,data_scientist 9 month_contract role 9 month_contract role within a key state gov child informat...


Inspect an example of the original, normalized and phrase-modelled text.

In [7]:
for col in ['text', 'normed_text', 'phrased_text']:
    print(dfs['data_scientist'].iloc[5][col][:250] + '\n')
    print('=' * 50)

The Opportunity

We're looking for an experienced practitioner with a track record in building and delivering practical Machine Learning driven products at scale. You will join an established Machine Learning / Data Science capability driving the glo

the opportunity -pron- be look for an experienced practitioner with a track record in build and deliver practical machine learning drive product at scale . -pron- will join an established machine learning / data science capability drive the global se

the opportunity -pron- be_look_for an_experienced practitioner with a track_record in build and deliver practical machine_learning drive product at_scale -pron- will join an established machine_learning data_science capability drive the global seek a



## 2. Get document frequency & lift
We'll now compute the % of jobs in each job class that each term appears in.

To ensure the same vocabulary appears across all job types, we'll first concatenate the dataframes together to derive the vocabulary from the full corpus.

In [8]:
# Add a column noting which type of job every job comes from before concatenating
for key, df in dfs.items():
    df['job_type'] = key

full_df = pd.concat(dfs.values(), sort=False)

Instantiate count vectorizer. Have already lowercased and fit phrase model, so can just use "unigrams". Can also drop stopwords here. Note that we tokenize on a single space, as the default tokenizer also splits on punctuation, meaning that phrases like "3_+\_year" get broken

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vectorizer = CountVectorizer(
    lowercase=False,
    tokenizer=lambda x: x.split(' '),
    stop_words='english',
    ngram_range=(1, 1),
    min_df=3
)

term_counts = vectorizer.fit_transform(full_df['phrased_text'])
print("Job descriptions: ")
term_counts.shape

Job descriptions: 


(182, 2237)

Or can use the scikit n-grams instead of phrase model. Gives similar results, with ordering a little shifted and some pros and cons. The main difference is it double counts subsets of larger N-grams, which is good and bad. E.g. 'Python' gets a very high score under the scikit method as every instance is counted, weheras Gensim learns two phrases including it: "python_r" and "r_python" so it can only get counted once in each of these.

In [11]:
# vectorizer = CountVectorizer(
#     lowercase=False,
#     stop_words='english',
#     ngram_range=(1, 3),
#     min_df=3
# )

# term_counts = vectorizer.fit_transform(full_df['normed_text'])
# term_counts.shape

Put it into a term frequency DataFrame

In [12]:
term_freq_df = pd.DataFrame(
    data=term_counts.toarray(),
    columns=vectorizer.get_feature_names(),
    index=full_df['job_type']
).drop('-pron-', axis=1)

term_freq_df.head()

,$,'s,+,+61,-_edge,-_making,-_minded,-_orient,-_term,/_life_balance,02,03,08,1,10,"10,000",100,10_year,12,12_month,13,15,2,2012,2017,2018,2019,2020,24,2_year,3,30,3_+_year,3_year,4,40,4_+_year,5,50,5_+_year,5_year,6,6_month,7,70,7370,8,8637,9,a_/_b,a_difference,a_few,a_range_of,a_variety_of,a_wide_range,ability,ability_to,able_to,aboriginal,about_us,academic,accelerate,accept,access,accessibility,accessible,accomplish,accord,accordance,account,accountability,accountable,accounting,accuracy,accurate,accurately,achieve,acquire,acquisition,across_a_range,across_multiple,act,act_as,action,actionable,actionable_insight,active,actively,activity,actuarial,ad,ad_-_hoc,adapt,add,add_value,additional,address,adept,administration,adobe,...,use,useful,user,utilise,utility,utilize,ux,vacancy,valid,validate,validation,valuable,value,varied,variety_of,various,vary,vast,vendor,verbal,verbal_communication_skill,verify,version,vertical,very_strong,veteran,vibrant,victoria,victorian,video,view,visa,vision,visit,visual,visualisation,visualisation_tool,visualise,visualization,visualization_tool,vital,voice,volume,volunteer,want,warehouse,warehousing,water,way,web,website,week,weekly,welcome,wellbeing,wellness,west,whilst,who_can,who_enjoy,who_want,wide,wide_variety,will_also,will_be_responsible,willing,willingness,win,windows,woman,word,work,work180,work_alongside,work_closely_with,work_collaboratively,work_with,workflow,workforce,working,working_environment,working_right,workload,workplace,workshop,world,worldwide,would_be,would_love,write,write_communication_skill,writing,written,xgboost,xml,year,yes,young,|,’s
job_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
data_engineer,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1
data_engineer,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8,0,0,1,1,3,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,3
data_engineer,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
data_engineer,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,4,0,0,0,0,3,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
data_engineer,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,1


### 2.1 Detour - combine similar phrases
There are several phrases and terms that are very similar and which we should combine, e.g. "statistic" and "statistics".

We can find these in a couple of manners:

1. Levenshtein distance (edit distance)
2. Vector cosine similarity

We will investigate both.

In [13]:
# top_terms = term_freq_df.sum().sort_values(ascending=False)
# top_terms.head(10)

In [14]:
# %%time
# import editdistance

# distances = []

# # Dynamic programming algorithm a little slow, so just consider top N terms
# TOP_N = 400

# for i in range(TOP_N):
#     for j in range(i + 1, TOP_N):
#         # Only compare words with same first letter;
#         # Otherwise get a huge number of irrelevant candidates
#         if top_terms.index[i][0] == top_terms.index[j][0]:
#             doc1 = nlp(top_terms.index[i].replace('_', ' '))
#             doc2 = nlp(top_terms.index[j].replace('_', ' '))
#             distances.append((doc1.text, doc2.text, editdistance.eval(doc1.text, doc2.text)))

# distance_df = pd.DataFrame(
#     data=sorted(distances, key=lambda kv: kv[2]),
#     columns=['term1', 'term2', 'distance']
# )
# # Number of irrelevant matches explodes beyond edit distance of 2
# distance_df = distance_df[distance_df['distance'] < 3]
# print(distance_df.shape)

In [15]:
# %%time
# similar_vecs = []

# docs = list(nlp.pipe([term.replace('_', ' ') for term in top_terms.index[:1000]]))

# for i in range(len(docs)):
#     doc1 = docs[i]
#     if doc1.vector_norm == 0:  # 0-vector for out-of-vocab terms
#         continue
#     for j in range(i + 1, len(docs)):
#         doc2 = docs[j]
#         if doc2.vector_norm == 0:
#             continue
#         else:
#             similar_vecs.append((doc1.text, doc2.text, doc1.similarity(doc2)))

# similarity_df = pd.DataFrame(
#     data=sorted(similar_vecs, key=lambda kv: kv[2], reverse=True),
#     columns=['term1', 'term2', 'similarity']
# )
# similarity_df = similarity_df[similarity_df['similarity'] > 0.8]

In [16]:
mappings = {
    'analytic': 'analytics',
    'analysis': 'analytics',
    'statistic': 'statistics',
    'statistical': 'statistics',
    'datum': 'data',
    'big_datum': 'big_data',
    'datum_science': 'data_science',
    'optimization': 'optimisation',
    'mentorship': 'mentoring',
    'mathematic': 'mathematics',
    'mathematical': 'mathematics',
    'visualization': 'visualisation',
    'machine_learning_technique': 'machine_learning',
    'machine_learning_algorithm': 'machine_learning',
    'machine_learn': 'machine_learning',
    'modeling': 'modelling',
    'predictive_model': 'predictive_modelling',
    'ai': 'artificial_intelligence',
}

In [17]:
def merge_terms(df, mapping, verbose=True):
    """For a given term frequency dataframe, merge terms according to a dictionary mapping"""
    for term1, term2 in mapping.items():
        if verbose:
            print(f"Map `{term1}` to `{term2}")
        df[term2] = df[term1] + df[term2]
        df = df.drop(term1, axis=1)
    return df

In [18]:
term_freq_df = merge_terms(term_freq_df, mappings)
term_freq_df.shape

Map `analytic` to `analytics
Map `analysis` to `analytics
Map `statistic` to `statistics
Map `statistical` to `statistics
Map `datum` to `data
Map `big_datum` to `big_data
Map `datum_science` to `data_science
Map `optimization` to `optimisation
Map `mentorship` to `mentoring
Map `mathematic` to `mathematics
Map `mathematical` to `mathematics
Map `visualization` to `visualisation
Map `machine_learning_technique` to `machine_learning
Map `machine_learning_algorithm` to `machine_learning
Map `machine_learn` to `machine_learning
Map `modeling` to `modelling
Map `predictive_model` to `predictive_modelling
Map `ai` to `artificial_intelligence


(182, 2218)

### 2.2 Back to the cleaning

Convert from term frequency to document probability - i.e. the percentage of documents containing each term per job category. To avoid 0-values, we apply Laplace smoothing by effectively adding an extra imaginary job description that contains each term for each job class.

In [19]:
def get_term_probability(term_freqs_df, smoothing=1):
    """Derive document frequency for each term then convert to smoothed probability"""
    # Binarise to yes/no for whether each term was in each document
    doc_freq_df = term_freqs_df.applymap(lambda x: min(x, 1))

    # Get smoothed probability for each job class
    stats_df = doc_freq_df.groupby(doc_freq_df.index).apply(
        lambda x: (x.sum() + 1) / (x.shape[0] + 1)
    )

    return stats_df

def get_lift(term_prob_df, control_grp='control'):
    """Derive uplift for each term compared to some control"""
    term_prob_df.index.name = 'term'
    for category in term_prob_df.index.values:
        if category != control_grp:
            term_prob_df.loc[f'{category}_lift'] = term_prob_df.loc[category] / term_prob_df.loc[control_grp]

    return term_prob_df.transpose().sort_values('control', ascending=False)

In [20]:
stats_df = get_term_probability(term_freq_df)
stats_df

,$,'s,+,+61,-_edge,-_making,-_minded,-_orient,-_term,/_life_balance,02,03,08,1,10,"10,000",100,10_year,12,12_month,13,15,2,2012,2017,2018,2019,2020,24,2_year,3,30,3_+_year,3_year,4,40,4_+_year,5,50,5_+_year,5_year,6,6_month,7,70,7370,8,8637,9,a_/_b,a_difference,a_few,a_range_of,a_variety_of,a_wide_range,ability,ability_to,able_to,aboriginal,about_us,academic,accelerate,accept,access,accessibility,accessible,accomplish,accord,accordance,account,accountability,accountable,accounting,accuracy,accurate,accurately,achieve,acquire,acquisition,across_a_range,across_multiple,act,act_as,action,actionable,actionable_insight,active,actively,activity,actuarial,ad,ad_-_hoc,adapt,add,add_value,additional,address,adept,administration,adobe,...,usage,use,useful,user,utilise,utility,utilize,ux,vacancy,valid,validate,validation,valuable,value,varied,variety_of,various,vary,vast,vendor,verbal,verbal_communication_skill,verify,version,vertical,very_strong,veteran,vibrant,victoria,victorian,video,view,visa,vision,visit,visual,visualisation,visualisation_tool,visualise,visualization_tool,vital,voice,volume,volunteer,want,warehouse,warehousing,water,way,web,website,week,weekly,welcome,wellbeing,wellness,west,whilst,who_can,who_enjoy,who_want,wide,wide_variety,will_also,will_be_responsible,willing,willingness,win,windows,woman,word,work,work180,work_alongside,work_closely_with,work_collaboratively,work_with,workflow,workforce,working,working_environment,working_right,workload,workplace,workshop,world,worldwide,would_be,would_love,write,write_communication_skill,writing,written,xgboost,xml,year,yes,young,|,’s
job_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
control,0.264706,0.411765,0.058824,0.029412,0.029412,0.058824,0.058824,0.029412,0.088235,0.147059,0.058824,0.058824,0.058824,0.088235,0.058824,0.058824,0.058824,0.029412,0.029412,0.117647,0.029412,0.117647,0.176471,0.058824,0.088235,0.088235,0.235294,0.029412,0.029412,0.029412,0.117647,0.088235,0.029412,0.147059,0.205882,0.058824,0.088235,0.147059,0.029412,0.058824,0.088235,0.058824,0.058824,0.088235,0.058824,0.029412,0.058824,0.029412,0.088235,0.058824,0.117647,0.058824,0.235294,0.058824,0.117647,0.117647,0.735294,0.205882,0.088235,0.029412,0.117647,0.029412,0.088235,0.088235,0.058824,0.058824,0.029412,0.029412,0.029412,0.088235,0.088235,0.058824,0.117647,0.058824,0.029412,0.029412,0.147059,0.058824,0.088235,0.117647,0.176471,0.088235,0.088235,0.117647,0.029412,0.029412,0.088235,0.176471,0.294118,0.029412,0.088235,0.058824,0.088235,0.029412,0.088235,0.176471,0.117647,0.088235,0.147059,0.117647,...,0.029412,0.441176,0.029412,0.117647,0.029412,0.029412,0.029412,0.088235,0.029412,0.029412,0.029412,0.029412,0.058824,0.294118,0.176471,0.088235,0.117647,0.058824,0.029412,0.058824,0.058824,0.117647,0.029412,0.058824,0.029412,0.058824,0.088235,0.088235,0.088235,0.088235,0.058824,0.176471,0.029412,0.147059,0.147059,0.029412,0.029412,0.029412,0.029412,0.029412,0.117647,0.117647,0.058824,0.029412,0.235294,0.029412,0.029412,0.029412,0.352941,0.088235,0.117647,0.147059,0.058824,0.176471,0.088235,0.088235,0.058824,0.147059,0.117647,0.029412,0.058824,0.147059,0.029412,0.176471,0.176471,0.058824,0.058824,0.088235,0.088235,0.058824,0.117647,0.882353,0.029412,0.058824,0.205882,0.147059,0.470588,0.058824,0.088235,0.205882,0.058824,0.029412,0.088235,0.147059,0.088235,0.264706,0.058824,0.147059,0.029412,0.235294,0.117647,0.117647,0.058824,0.029412,0.029412,0.382353,0.029412,0.058824,0.029412,0.294118
data_engineer,0.088235,0.264706,0.176471,0.058824,0.058824,0.058824,0.058824,0.176471,0.058824,0.058824,0.029412,0.117647,0.029412,0.088235,0.058824,0.029412,0.088235,0.029412,0.029412,0.058824,0.029412,0.058824,0.088235,0.058824,0.029412,0.029412,0.176471,0.029412,0.029412,0.088235,0.058824,0.058824,0.117647,0.058824,0.117647,0.058824,0.058824,0.088235,0.058824,0

In [21]:
# Add in the lift and transpose to get long not wide dataframe
stats_df = get_lift(stats_df)
stats_df.head(10)

term,control,data_engineer,data_scientist,data_engineer_lift,data_scientist_lift
experience,0.882353,0.941176,0.931624,1.066667,1.055840
work,0.882353,0.823529,0.871795,0.933333,0.988034
team,0.794118,0.911765,0.854701,1.148148,1.076290
provide,0.764706,0.529412,0.512821,0.692308,0.670611
role,0.764706,0.764706,0.623932,1.000000,0.815911
include,0.735294,0.558824,0.598291,0.760000,0.813675
ability_to,0.735294,0.558824,0.504274,0.760000,0.685812
opportunity,0.735294,0.558824,0.538462,0.760000,0.732308
strong,0.647059,0.588235,0.495726,0.909091,0.766123
people,0.647059,0.500000,0.282051,0.772727,0.435897


In [22]:
from sklearn.preprocessing import MinMaxScaler

for col in ['data_scientist', 'data_engineer']:
    input_lift = stats_df[f'{col}_lift'].apply(
        lambda x: max(1, x)
    ).values
    scaler = MinMaxScaler()
    stats_df[f'{col}_lift_scaled'] = scaler.fit_transform(input_lift.reshape(-1, 1))
    stats_df[f'{col}_score'] = stats_df[[col, f'{col}_lift_scaled']].mean(axis=1)

In [23]:
# Save to file
stats_df.to_csv(PARAMS['CLEANED_PATH'] / 'stats.csv', index=None)

## 3. Top skills by document frequency

For data science:

In [24]:
stats_df.sort_values('data_scientist_lift', ascending=False).drop('data_science').head(25)

term,control,data_engineer,data_scientist,data_engineer_lift,data_scientist_lift,data_scientist_lift_scaled,data_scientist_score,data_engineer_lift_scaled,data_engineer_score
python,0.029412,0.676471,0.777778,23.0,26.444444,1.000000,0.888889,1.000000,0.838235
machine_learning,0.029412,0.352941,0.769231,12.0,26.153846,0.988579,0.878905,0.500000,0.426471
statistics,0.029412,0.147059,0.615385,5.0,20.923077,0.783003,0.699194,0.181818,0.164439
r,0.029412,0.176471,0.615385,6.0,20.923077,0.783003,0.699194,0.227273,0.201872
mathematics,0.029412,0.058824,0.401709,2.0,13.658120,0.497481,0.449595,0.045455,0.052139
spark,0.029412,0.500000,0.282051,17.0,9.589744,0.337588,0.309820,0.727273,0.613636
sql,0.058824,0.676471,0.538462,11.5,9.153846,0.320457,0.429459,0.477273,0.576872
predictive_modelling,0.029412,0.029412,0.247863,1.0,8.427350,0.291905,0.269884,0.000000,0.014706
scala,0.029412,0.294118,0.230769,10.0,7.846154,0.269063,0.249916,0.409091,0.351604
java,0.029412,0.323529,0.222222,11.0,7.555556,0.257642,0.239932,0.454545,0.389037


Data engineering:

In [25]:
stats_df.sort_values('data_engineer_lift', ascending=False).drop('data_engineer').head(25)

term,control,data_engineer,data_scientist,data_engineer_lift,data_scientist_lift,data_scientist_lift_scaled,data_scientist_score,data_engineer_lift_scaled,data_engineer_score
python,0.029412,0.676471,0.777778,23.0,26.444444,1.000000,0.888889,1.000000,0.838235
spark,0.029412,0.500000,0.282051,17.0,9.589744,0.337588,0.309820,0.727273,0.613636
etl,0.029412,0.441176,0.068376,15.0,2.324786,0.052066,0.060221,0.636364,0.538770
processing,0.029412,0.411765,0.102564,14.0,3.487179,0.097749,0.100157,0.590909,0.501337
hadoop,0.029412,0.411765,0.170940,14.0,5.811966,0.189117,0.180028,0.590909,0.501337
data_pipeline,0.029412,0.382353,0.042735,13.0,1.452991,0.017803,0.030269,0.545455,0.463904
machine_learning,0.029412,0.352941,0.769231,12.0,26.153846,0.988579,0.878905,0.500000,0.426471
data_lake,0.029412,0.352941,0.017094,12.0,0.581197,0.000000,0.008547,0.500000,0.426471
sql,0.058824,0.676471,0.538462,11.5,9.153846,0.320457,0.429459,0.477273,0.576872
big_data,0.058824,0.647059,0.273504,11.0,4.649573,0.143433,0.208469,0.454545,0.550802


## 4. Top skills required on LinkedIn
For LinkedIn, job advertisers can list explicit skills they're looking for. Count these up.

In [26]:
from collections import Counter

def parse_comma_separated_list_string(string):
    """
    Convert a comma-separated list of skills to a true list,
    treating commas within quotation marks as not separators
    """
    start_idx = 0
    in_quotes = False
    items = set()
    
    for i, char in enumerate(string):
        if char in {'"', '“', '”'}:
            in_quotes = not in_quotes
        elif char == ',':
            if not in_quotes:
                # Found a skill separator
                item = string[start_idx:i].strip()
                item = item.replace('"', '').replace('“', '').replace('”', '')
                items.add(item)
                start_idx = i + 1
    item = string[start_idx:].strip()
    items.add(item.replace('"', '').replace('“', '').replace('”', ''))
    return items

def get_skills_df(skills_series):
    """Take a series of skills and return a DataFrame indicating counts and probs"""
    skills_series = skills_series[skills_series.notna()]
    skillsets = skills_series.apply(parse_comma_separated_list_string)
    
    counter = Counter()
    for skillset in skillsets:
        counter.update(skillset)

    skills_df = pd.DataFrame(
        data=counter.most_common(),
        columns=['skill', 'count']
    ).sort_values('count', ascending=False)
    skills_df['fraction'] = skills_df['count'] / skillsets.shape[0]
    return skills_df

def analyse_skill_reqs(required_skills, applicant_skills):
    """Assess skills explicitly called out against those offered by applicant"""
    required_skills_df = get_skills_df(required_skills)
    applicant_skills_df = get_skills_df(applicant_skills)

    merge_df = required_skills_df.merge(
        applicant_skills_df, on='skill', how='outer',
        suffixes=('_required', '_applicant'),
        validate='one_to_one'
    )

    merge_df['delta'] = merge_df['fraction_applicant'] - merge_df['fraction_required']
    return merge_df.sort_values('fraction_required', ascending=False)

In [27]:
datasci_skills_df = analyse_skill_reqs(dfs['data_scientist']['required skills'],
                                       dfs['data_scientist']['applicants skills'])

print(datasci_skills_df.shape)
datasci_skills_df.head(10)

(136, 6)


,skill,count_required,fraction_required,count_applicant,fraction_applicant,delta
0,Data Science,18.0,0.857143,17.0,0.500000,-0.357143
1,Machine Learning,11.0,0.523810,30.0,0.882353,0.358543
2,Data Mining,9.0,0.428571,11.0,0.323529,-0.105042
3,Python (Programming Language),8.0,0.380952,32.0,0.941176,0.560224
4,Analytics,8.0,0.380952,3.0,0.088235,-0.292717
5,Data Analytics,8.0,0.380952,1.0,0.029412,-0.351541
6,R,7.0,0.333333,30.0,0.882353,0.549020
7,Analytical Skills,6.0,0.285714,NaN,NaN,NaN
8,Statistical Modeling,5.0,0.238095,NaN,NaN,NaN
10,Problem Solving,4.0,0.190476,NaN,NaN,NaN


In [28]:
datasci_skills_df.to_csv(PARAMS['CLEANED_PATH'] / 'datasci_skills.csv', index=None)

In [29]:
dataeng_skills_df = analyse_skill_reqs(dfs['data_engineer']['required skills'],
                                       dfs['data_engineer']['applicants skills'])

dataeng_skills_df.head(10)

,skill,count_required,fraction_required,count_applicant,fraction_applicant,delta
0,Amazon Web Services (AWS),5.0,0.625,2.0,0.222222,-0.402778
2,Analytics,4.0,0.500,NaN,NaN,NaN
3,"Extract, Transform, Load (ETL)",4.0,0.500,3.0,0.333333,-0.166667
1,Apache Spark,4.0,0.500,2.0,0.222222,-0.277778
4,American Welding Society (AWS),3.0,0.375,NaN,NaN,NaN
12,Scripting,2.0,0.250,NaN,NaN,NaN
18,Data Mining,2.0,0.250,NaN,NaN,NaN
17,Apache Kafka,2.0,0.250,NaN,NaN,NaN
15,Big Data,2.0,0.250,3.0,0.333333,0.083333
14,PySpark,2.0,0.250,NaN,NaN,NaN


In [30]:
dataeng_skills_df.to_csv(PARAMS['CLEANED_PATH'] / 'dataeng_skills.csv', index=None)